<a href="https://colab.research.google.com/github/yu0ki/BERT_Practice/blob/main/Chapter8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
'''
本章でやること：固有表現抽出
つまり文章から人名、組織名といったものを抽出する
まずは簡単な方法を学んで、次にBERTを使ったものを学んでいく
'''

'\n本章でやること：固有表現抽出\nつまり文章から人名、組織名といったものを抽出する\nまずは簡単な方法を学んで、次にBERTを使ったものを学んでいく\n'

In [3]:
# まずはディレクトリ移動
!mkdir chap8
%cd ./chap8


mkdir: cannot create directory ‘chap8’: File exists
/content/chap8


In [4]:
# 次はライブラリ
# バージョンが教科書のだと古いようなので、模範解答（https://github.com/stockmarkteam/bert-book/blob/master/Chapter6.ipynb）にバージョンを合わせている
!pip install transformers==4.18.0 fugashi==1.1.0 ipadic==1.0.0 pytorch-lightning==1.6.1

# 集合に関する数学的な操作のライブラリかな？
# https://qiita.com/anmint/items/37ca0ded5e1d360b51f3
import itertools

import random
import json
from tqdm import tqdm
import numpy as np

# unicode文字列に対していろいろ操作できそうなライブラリ
# https://docs.python.org/ja/3/library/unicodedata.html
import unicodedata

import torch
from torch.utils.data import DataLoader
from transformers import BertJapaneseTokenizer, BertForTokenClassification
import pytorch_lightning as pl

# 日本語学習済みモデル
MODEL_NAME = 'cl-tohoku/bert-base-japanese-whole-word-masking'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
'''
固有表現抽出とは

人名・組織名などの固有名詞
日付などの時間表現
金額などの数値表現

などを抽出する。そして、それが人名なのかなんなのか、カテゴリを判定する

ちなみに、どんなカテゴリが定義されているかは何パターンかあり、問題によって変える
IREX
拡張固有表現階層
などがある
'''

'\n固有表現抽出とは\n\n人名・組織名などの固有名詞\n日付などの時間表現\n金額などの数値表現\n\nなどを抽出する。そして、それが人名なのかなんなのか、カテゴリを判定する\n\nちなみに、どんなカテゴリが定義されているかは何パターンかあり、問題によって変える\nIREX\n拡張固有表現階層\nなどがある\n'

In [6]:
'''
表記揺れについて

固有表現には全角・半角などの違いで、表記揺れが発生する
よって、これらを正規化することで１つに統一する必要がある

正規化には、unicodedata.normalize('NFKC', text) が用いられる
NFKCは正規化のモード
textは正規化対象の文字列

'''
# 匿名関数lambda
normalize = lambda s: unicodedata.normalize("NFKC", s)

# 全角ABCを半角に正規化
print(f'ＡＢＣ　-> {normalize("ＡＢＣ")}')
# 半角ABCを半角に正規化
print(f'ABC　-> {normalize("ABC")}')

# 全角１２３を半角に正規化
print(f'１２３　-> {normalize("１２３")}')
# 半角ABCを半角に正規化
print(f'123　-> {normalize("123")}')

# 全角カタカナを全角に正規化
print(f'アイウ　-> {normalize("アイウ")}')
# 半角カタカナを全角に正規化
print(f'ｱｲｳ　-> {normalize("ｱｲｳ")}')

ＡＢＣ　-> ABC
ABC　-> ABC
１２３　-> 123
123　-> 123
アイウ　-> アイウ
ｱｲｳ　-> アイウ


In [7]:
'''
固有表現抽出方法１
IO法

簡単なアルゴリズムであるが、一般的にはBIO法の方がよく使われる
    IO法でもまあまあいいが、一部対応できない表現があるためBIOが使われる傾向にある

アルゴリズム
１。トークンが固有表現の一部であれば、トークンにタグ「I-(Type)」をつける.  ex) I - (人名)　など
２。トークンが固有表現の一部でないならば、トークンのタグはOとする
3。O以外の同じタグが連続している部分トークン列を連結して、固有表現とする
    トークナイザがトークン分割の際に「##」を付与している場合があるので、これは除外して結合する

I は Inside, O は Outsideから来ている


問題点
同じタグを持つ固有名詞が複数続いた場合は、全てが連結されて一つの固有表現として抽出される
日米　などは日本とアメリカで別々のはずなのに、1個に合体してしまう

'''

'\n固有表現抽出方法１\nIO法\n\n簡単なアルゴリズムであるが、一般的にはBIO法の方がよく使われる\n    IO法でもまあまあいいが、一部対応できない表現があるためBIOが使われる傾向にある\n\nアルゴリズム\n１。トークンが固有表現の一部であれば、トークンにタグ「I-(Type)」をつける.  ex) I - (人名)\u3000など\n２。トークンが固有表現の一部でないならば、トークンのタグはOとする\n3。O以外の同じタグが連続している部分トークン列を連結して、固有表現とする\n    トークナイザがトークン分割の際に「##」を付与している場合があるので、これは除外して結合する\n\nI は Inside, O は Outsideから来ている\n\n\n問題点\n同じタグを持つ固有名詞が複数続いた場合は、全てが連結されて一つの固有表現として抽出される\n日米\u3000などは日本とアメリカで別々のはずなのに、1個に合体してしまう\n\n'

In [28]:
# 固有表現では普段のトークンにタグを付与するので
# それに対応したトークナイザを定義しないといけない

# そのトークナイザの中では、学習・推論に使用する関数を定義
# ここは最悪あんまり理解してなくてもいいらしい


# BertJapaneseTokenizerを継承：つまり、selfはBertJapaneseTokenizerの関数は使える
class NER_tokenizer(BertJapaneseTokenizer):

    '''
    文章と、それに含まれる固有表現が与えられたときに、符号化とラベル列の作成を行う関数
    この関数の出力は、BERTモデルに入力できる形になっている。（学習時に使用する関数）
    つまり、固有表現に適切なラベルを付与してBERTに入力できるように変換する関数
    これをBERTに入力すると、ラベルがついているため、損失を返してくれる
    これをもとにパラメータを学習する
    '''
    def encode_plus_tagged(self, text, entities, max_length):

        # 手順１：固有表現の前後でtextを分割し、それぞれのラベルをつけておく

        # entities は配列
        # 配列のようそはdict型
        # {'name' : 固有表現, 'span': [文章中の文字列の開始位置index, 終了index+1], 'type' : 固有表現のラベル, 'type_id' : typeに紐づく数字（１：１対応）}
        # sorted :https://note.nkmk.me/python-list-sort-sorted/
        # 単語の文章中での開始位置でソート
        entities = sorted(entities, key=lambda x: x['span'][0])

        # 分割後の文字列を追加する
        splitted = []

        # 次にラベルをつけるのはどこからか？を表す
        position = 0

        '''
        entitiesの中には、文章中に入っている固有表現とそのラベル一覧が入っている（教師データ）
        textを普段通り分割すると、固有表現が途中で分断されてしまって、教師データと一致する固有表現が検出できなくなってしまうことがある
        これを回避するために、以下のfor文では教師データの固有表現を見つけては、textを分割するという対応をとっている
        分割ついでにラベルもちゃんと記録する
        '''
        for entity in entities:
            start = entity['span'][0]
            end = entity['span'][1]

            # 固有表現のラベル
            label = entity['type_id']

            # 固有表現でないものにはラベルOをつける
            splitted.append({'text' : text[position:start], 'label' : 0})

            # 固有表現には対応するラベルをIDで付与
            splitted.append({'text' : text[start:end], 'label' : label})

            # 次にラベルをつけるのはend以降
            position = end

        # 最後余った文字列をラベル0にする
        splitted.append({'text' : text[position:], 'label' : 0})

        # 長さ０の文字列をsplittedから取り除く
        splitted = [ s for s in splitted if s['text']]

        # 手順２：分割されたそれぞれの文字列をトークン化し、ラベルづけする

        # トークンを入れる
        tokens = []
        # ラベルを入れる
        labels = []

        for text_splitted in splitted:
            text = text_splitted['text']
            label = text_splitted['label']

            # テキストをトークン化
            tokens_splitted = self.tokenize(text)
            # 対応するラベルを追加
            labels_splitted = [label] * len(tokens_splitted)

            tokens.extend(tokens_splitted)
            labels.extend(labels_splitted)

        
        # 手順３：符号化を行い、BERTに入力可能にする
        input_ids = self.convert_tokens_to_ids(tokens)

        # 公式ドキュメントが見つからないが
        # prepare_for_model はinput_idsを符号化するらしい
        # 公式これか？？https://huggingface.co/docs/transformers/internal/tokenization_utils#transformers.PreTrainedTokenizerBase.prepare_for_model

        encoding = self.prepare_for_model(
            input_ids,
            max_length=max_length,
            padding='max_length',
            truncation=True
        )


        # 特殊トークン[CLS], [SEP]のラベルを０にする
        labels = [0] + labels[:max_length-2] + [0]

        # [PAD]もラベルを０に
        # [PAD]は文末にあるので、その数だけ最後に追加すればいい
        labels = labels + [0]*(max_length - len(labels))

        encoding['labels'] = labels

        return encoding

    
    '''
    文章を符号化するとともに、各トークンの文章中の位置を特定する関数
    文章中に未知語などが含まれていた場合、先ほどの関数でラベルづけした出力（input_ids やそれに対応するlabel 元の文章は保存されない）からは
    元の文章を復元して、固有表現はこれだ！と特定できない。
    そのため、位置を特定しておく必要がある
    （推論時に使用する関数）
    '''
    def encode_plus_untagged(
        self, text, max_length=None, return_tensors=None
    ):
        # 手順１：文章のトークン化

        # トークンを追加していく
        tokens = []
        # トークンに対応する文章中の文字列を追加していく
        tokens_original = []

        # MeCabで単語に分割
        # https://huggingface.co/transformers/v4.11.3/_modules/transformers/models/bert_japanese/tokenization_bert_japanese.html
        # self.word_tokenizerにはBasicTokenizerというクラスのインスタンスが入ってるっぽい
        words = self.word_tokenizer.tokenize(text)
        # print('1')
        # 手順２：単語をサブワード分割して、元のテキストを保存
        for word in words:
            # 単語をサブワードに分割
            # 未知語も分割すれば対応可能になる可能性が上がるっぽいね　　　例）東京タワーが未知語でも、東京とタワーはそれぞれ既知の単語かもしれない
            # https://note.com/npaka/n/nb08941a36c8b#I3MMH
            tokens_word = self.subword_tokenizer.tokenize(word)
            tokens.extend(tokens_word)

            # 未知語への対応
            # 未知語に遭遇した場合は元のテキストを残しておかないと、全て[UNK]に変換されてしまって区別できなくなる
            if tokens_word[0] == '[UNK]':
                tokens_original.append(word)
            else:
                # 先頭の単語以外は##を除去して保存
                # 未知語対策のifを挟まないと、[UNK]が直で保存される
                tokens_original.extend([
                    token.replace('##', '') for token in tokens_word
                ])
        # print('2')
        # 手順３：各トークンの文章中での位置を保存（空白位置も考慮）
        position = 0
        spans = [] # トークンの位置情報を追加していく

        for token in tokens_original:
            # トークン長
            l = len(token)

            # text（元の入力文章）中の[position..position+l]を切り出して、tokenと一致するか確認
            # while 1 = while true
            while 1:
                # 一致しない場合
                if token != text[position:position+l]:
                    position += 1
                else:
                    spans.append([position, position+l])
                    position += 1
                    break

        # print('3')
        # 手順４：符号化を行い、BERT入力可能にする
        input_ids = self.convert_tokens_to_ids(tokens)

        # input_idsが既にわかっている場合に使用できる符号化
        # 確かめてはいないが、いつもはBertJapanezeTokenizerのインスタンスを関数として呼び出していたが、それができないため、わざわざこの関数を使用しているのではないのか？と思っている
        # selfに直接引数与えれるのかな・・・
        encoding = self.prepare_for_model(
            input_ids,
            max_length=max_length,
            padding='max_length' if max_length else False,
            truncation = True if max_length else False
        )

        # [CLS], [SEP], [PAD]の処理
        # 系列長
        sequence_length = len(encoding['input_ids'])

        # [CLS]に対応するダミーspan追加
        # sequence_lengthには[sep][cls]が入っているが、spansの時点ではこれらは入っていないデータを使っているので、数合わせのために-2
        # sequence_lenghtはspansの長さより長い
        spans = [[-1, -1]] + spans[:sequence_length-2]

        # sep, padに対する追加
        spans = spans + [[-1, -1]] * (sequence_length - len(spans))

        # print('4')
        
        # 手順５：必要に応じて出力の型を調整
        if return_tensors == 'pt':
            encoding = { k : torch.tensor([v]) for k, v in encoding.items() }
        
        return encoding, spans


    '''
    「文章」と「タグ列」と「各トークンの文章中の位置」が与えられた時に、文章中に含まれる固有表現に対応する文字列や位置を特定する関数
    文章、ラベル列の予測値、各トークンの位置から固有表現を得る（推論時に使用）
    '''
    def convert_bert_output_to_entities(self, text, labels, spans):
        # 手順１：labels, spansから特殊トークンに対応する部分を取り除く
        # ダミーのspanは-1から始まることを利用
        labels = [label for label, span in zip(labels, spans) if span[0] != -1]
        spans = [span for span in spans if span[0] != -1]

        # 手順２：同じラベルが連続するトークンをまとめて、固有表現を抽出
        entities = []

        for label, group \
            in itertools.groupby(enumerate(labels), key=lambda x: x[1]):

            # itertools groupby https://note.nkmk.me/python-itertools-groupby/
            group = list(group)
            start = spans[group[0][0]][0]
            end = spans[group[-1][0]][1]

            if label != 0: # ラベルが0以外ならば、新たな固有表現として追加。
                entity = {
                    "name": text[start:end],
                    "span": [start, end],
                    "type_id": label
                }
                entities.append(entity)

        return entities





In [29]:
# 上記のトークナイザーをインスタンス化
tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.


In [30]:
# encode_plus_taggedの挙動確認
text = '昨日のみらい事務所との打ち合わせは順調だった'

# ちなみにIO法のOタグは0(zero), I タグは１以上の数字で表される 
entities = [
    {'name' : 'みらい事務所', 'span' : [3, 9], 'type_id' : 1}
]

encoding = tokenizer.encode_plus_tagged(
    text, entities, max_length=20
)
print(encoding)

{'input_ids': [2, 10271, 28486, 5, 546, 10780, 2464, 13, 5, 1878, 2682, 9, 10750, 308, 10, 3, 0, 0, 0, 0], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0], 'labels': [0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}


In [31]:
# encode_plus_untaggedの挙動確認

text = '騰訊の英語名はTencent Holdings Ltdである。'
encoding, spans = tokenizer.encode_plus_untagged(
    text, return_tensors='pt'
)
print('# encoding')
print(encoding)
print('# spans')
print(spans)

# encoding
{'input_ids': tensor([[    2,     1, 26280,     5,  1543,   125,     9,  6749, 28550,  2953,
         28550, 28566, 21202, 28683, 14050, 12475,    12,    31,     8,     3]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
# spans
[[-1, -1], [0, 1], [1, 2], [2, 3], [3, 5], [5, 6], [6, 7], [7, 9], [9, 10], [10, 12], [12, 13], [13, 14], [15, 18], [18, 19], [19, 23], [24, 27], [27, 28], [28, 30], [30, 31], [-1, -1]]


In [32]:
# convert_bert_output_to_entitiesの挙動確認
# 適当なバートからの出力を置いておく
labels_predicted = [0, 1, 1, 0,0,0,0,1,1,1,1,1,1,1,1,1,0,0,0,0]

entities = tokenizer.convert_bert_output_to_entities(
    text, labels_predicted, spans
)
print(entities)

[{'name': '騰訊', 'span': [0, 2], 'type_id': 1}, {'name': 'Tencent Holdings Ltd', 'span': [7, 27], 'type_id': 1}]


In [33]:
'''
ここからは、BERTも使って固有表現抽出を行っていく
BertForTokenClassificationが使用できる
'''

tokenizer = NER_tokenizer.from_pretrained(MODEL_NAME)

# 固有表現のタイプが３つ　＋　Oタグでlabel数は4
# 1:組織名、２：人名、３：製品名
bert_tc = BertForTokenClassification.from_pretrained(
    MODEL_NAME, num_labels = 4
)

bert_tc = bert_tc.cuda()

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'BertJapaneseTokenizer'. 
The class this function is called from is 'NER_tokenizer'.
Some weights of the model checkpoint at cl-tohoku/bert-base-japanese-whole-word-masking were not used when initializing BertForTokenClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expecte

In [34]:
'''
推論時の使い方
'''

# データの定義
text = 'AさんはB大学に入学した。'

# 符号化を行い、トークンの文章中の位置も特定する
encoding, spans = tokenizer.encode_plus_untagged(
    text, return_tensors = 'pt'
)

encoding = { k : v.cuda() for k, v in encoding.items() }

# BERTでトークンごとの分類スコアを出力
# スコアが最も高いラベルを予測値とする

with torch.no_grad():
    output = bert_tc(**encoding)
    # scores[バッチサイズ, 系列長, ラベルの数]
    scores = output.logits
    labels_predicted = scores[0].argmax(-1).cpu().numpy().tolist()

# ラベルを固有表現に変換
entities = tokenizer.convert_bert_output_to_entities(text, labels_predicted, spans)
print(entities)


# ちなみに、チューニングしていないので出力は出鱈目である

[{'name': 'さん', 'span': [1, 3], 'type_id': 1}, {'name': 'は', 'span': [3, 4], 'type_id': 3}, {'name': '大学', 'span': [5, 7], 'type_id': 3}, {'name': 'に', 'span': [7, 8], 'type_id': 1}, {'name': '入学', 'span': [8, 10], 'type_id': 3}, {'name': 'した。', 'span': [10, 13], 'type_id': 1}]


In [39]:
'''
学習する際のコード

'''

# 学習用データの定義
data = [
    {
        'text': 'AさんはB大学に入学した。',
        'entities': [
            {'name': 'A', 'span': [0, 1], 'type_id': 2},
            {'name': 'B大学', 'span': [4, 7], 'type_id': 1}
        ]
    },
    {
        'text': 'CDE株式会社は新製品「E」を販売する。',
        'entities': [
            {'name': 'CDE株式会社', 'span': [0, 7], 'type_id': 1},
            {'name': 'E', 'span': [12, 13], 'type_id': 3}
        ]
    }
]

# 各データを符号化
max_length = 32

# 符号化結果を格納
dataset_for_loader = []

# 符号化
for sample in data:
    text = sample['text']
    entities = sample['entities']

    encoding = tokenizer.encode_plus_tagged(
        text, entities, max_length
    )
    encoding = { k : torch.tensor(v) for k, v in encoding.items() }
    dataset_for_loader.append(encoding)


# データローダを作成
dataloader = DataLoader(dataset_for_loader, batch_size = len(data))

# ミニバッチを取り出して損失を得る
for batch in dataloader:
    batch = { k : v.cuda() for k, v in batch.items() }
    output = bert_tc(**batch)
    loss = output.loss
